# definitions_testscript.ipynb

A script for testing and development of automatic generation of `ArchetypeBuildingModel.jl` definitions from AmBIENCe data.

In [ ]:
import ambience2abm as amb

### Read the raw data and assumptions.

ambience = amb.AmBIENCeDataset(
    interior_node_depth=0.1,
    period_of_variations=1209600
)

## First, let's quickly check the building fabrics and building nodes.

These are assumptions corresponding to the `ZonWallInt_B` RC-model in the AmBIENCe data
as closely as possible to define for `ArchetypeBuildingModel.jl`.

In [ ]:
### Test definitions processing

defs = amb.ABMDefinitions(
    ambience,
    aggregate_building_type=False, # First no aggregation
    aggregate_building_period=False
)
defs.data

In [ ]:
### Check given parameters

[
    defs.room_height_m,
    defs.weather_start,
    defs.weather_end,
    defs.partition_wall_length_ratio_to_external_walls_m_m,
    defs.window_area_thermal_bridge_surcharge_W_m2K
]

In [ ]:
### Check building fabrics

defs.building_fabrics

In [ ]:
### Check building nodes and structure types

defs.building_node__structure_type

Seems ok.

## Next, let's look at the archetype building definitions.

Each row of the raw AmBIENCe data corresponds to an archetype building model in
`ArchetypeBuildingModel.jl`, so let's examine the properties a bit.

In [ ]:
### Check building archetype params

archetype_data = defs.building_archetype()
archetype_data

In [ ]:
### Check if building frame depth values have `nan` due to complex sqrt

archetype_data[archetype_data["building_frame_depth_m"].isna()]

# Which should return an empty dataframe

In [ ]:
### Check mean values

cols = [
    "building_frame_depth_m",
    "number_of_storeys",
    "room_height_m",
    "window_area_to_external_wall_ratio_m2_m2",
    "reference_floor_area_m2",
    "reference_wall_area_m2",
    "reference_window_area_m2",
    "reference_roof_area_m2",
]

archetype_data[cols].mean()

In [ ]:
### Check median values

archetype_data[cols].median()

On average, the parameters seem reasonable enough,
although average `building_frame_depth_m` less than 8-6 metres seems a bit low-ish from a Finnish point-of-view.

In [ ]:
### Check maximum values

archetype_data[cols].max()

The maximum values don't seem entirely unreasonable either,
with the possible exception of a 71 m maximum building frame depth.
Fortunately, it would seem that excessive building frame depths mostly occur
in non-residential buildings, and large warehouses could reasonably have shapes like that.

In [ ]:
archetype_data[archetype_data["building_frame_depth_m"] >= 30]

In [ ]:
### Check minimum values

archetype_data[cols].min()

However, the minimum values get pretty unrealistic.
Cyprus has some clearly unreasonable data, with less than 1 m2 of floor area per storey. There are also several buildings with ground floor areas less than 10 m2,
which I also find quite suspicious.

In [ ]:
ambience.data[ambience.data["REFERENCE BUILDING GROUND FLOOR AREA (m2)"] <= 10.0]

## Forming `building_scope`

The `building_scope` objects define how the building stock statistics are aggregated
into the properties of the desired archetype buildings.
For replicating the reference buildings in the AmBIENCe data as close ly as possible,
the heat sources are re-aggregated, but the material combinations neglected.

In [ ]:
### Check building scopes

defs.building_scope()

In [ ]:
### Check building scope to building type mapping

defs.building_scope__building_type()

In [ ]:
### Check building scope to heat source mapping

defs.building_scope__heat_source()

In [ ]:
### Check building scope to location ID mapping

defs.building_scope__location_id()

## Test export and data packaging

Similar to the building stock data, the definitions can be packaged to make them
easier to deal with in Spine Toolbox.
This functionality is tested below.*

In [ ]:
### Test data package export

defs.export_csvs()
pkg = defs.create_datapackage()
pkg

## Test definition aggregation

The above definition tests were carried out without any automatic aggregation.
However, in practise, this dataset shouldn't be used without any aggregation
due to the considerable uncertainties and oddities in the raw inputs.

For convenience, aggregation over construction periods and building types is provided.
Let's test that it actually works as intended.

In [ ]:
### Check aggregated definitions processing

agg_defs = amb.ABMDefinitions(
    ambience,
    aggregate_building_type=True,
    aggregate_building_period=True,
)
agg_defs.data

In [ ]:
### Check aggregated building archetypes

agg_archetype_data = agg_defs.building_archetype()
agg_archetype_data

In [ ]:
### First off, there should be less aggregated archetypes than non-aggregated ones

archetype_data.size >= agg_archetype_data.size

In [ ]:
### Let's check bounds for archetype parameters

cols = [
    "building_frame_depth_m",
    "number_of_storeys",
    "room_height_m",
    "window_area_to_external_wall_ratio_m2_m2",
    "partition_wall_length_ratio_to_external_walls_m_m",
    "window_area_thermal_bridge_surcharge_W_m2K",
    "reference_floor_area_m2",
    "reference_wall_area_m2",
    "reference_window_area_m2",
    "reference_roof_area_m2"
]
archetype_data["location_id"] = archetype_data["building_scope"].str[:2]
agg_archetype_data["location_id"] = agg_archetype_data["building_scope"].str[:2]
archetype_bounds = archetype_data.groupby("location_id").agg(
    {col: ["min", "max"] for col in cols}
)
archetype_bounds


In [ ]:
### Compare these to aggregated archetype building bounds

agg_archetype_bounds = agg_archetype_data.groupby("location_id").agg(
    {col: ["min", "max"] for col in cols}
)
agg_archetype_bounds

In [ ]:
### The aggregated bounds should be within the non-aggregated bounds.

all(
    [
        all(archetype_bounds[:]["min"] <= agg_archetype_bounds[:]["min"]),
        all(archetype_bounds[:]["max"] >= agg_archetype_bounds[:]["max"])
    ]
)

Aggregation seems to work as intended.
Some of the most insane archetype parameters get tempered a bit by the aggregation
as a bonus.